# CDR Analysis 

In [5]:
import os
os.environ["HADOOP_HOME"] = "/opt/hadoop"
os.environ["PATH"] = os.environ["PATH"] + ":/opt/hadoop/bin"
os.environ["HADOOP_CONF_DIR"] = "/hadoop/etc/hadoop"

In [2]:
from pyspark.sql import SparkSession

# Create Spark session with HDFS configuration
spark = SparkSession.builder \
    .appName("CDR-Analysis") \
    .config("spark.master", "spark://spark-master:7077") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://namenode:9000") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g") \
    .getOrCreate()

# Print Spark configuration for troubleshooting
print("Spark version:", spark.version)
print("\nSpark configuration:")
for item in spark.sparkContext.getConf().getAll():
    print(f"  {item[0]}: {item[1]}")

# Check if we can access HDFS through Spark
try:
    # List files in HDFS
    files = spark.sparkContext._jvm.org.apache.hadoop.fs.FileSystem.get(
        spark.sparkContext._jsc.hadoopConfiguration()).listStatus(
        spark.sparkContext._jvm.org.apache.hadoop.fs.Path("/"))
    
    print("\nHDFS root directory contents:")
    for file in files:
        print(f"  {file.getPath().getName()}")
        
except Exception as e:
    print(f"\nError accessing HDFS through Spark: {e}")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/07 19:04:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/07 19:04:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/05/07 19:04:07 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


Spark version: 3.5.1

Spark configuration:
  spark.executor.memory: 2g
  spark.driver.extraJavaOptions: -Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/jdk.internal.ref=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false
  spark.hadoop.fs.defaultFS: hdfs://name

In [8]:
import pyspark
print(pyspark.__version__)

3.5.5


In [11]:
!pip install findspark

In [3]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

# Create Spark session with HDFS configuration
spark = SparkSession.builder \
    .appName("CDR-Analysis") \
    .config("spark.master", "spark://spark-master:7077") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://namenode:9000") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g") \
    .getOrCreate()

print("Spark version:", spark.version)

Spark version: 3.5.1


In [4]:
# List files in HDFS root directory using Spark
try:
    # Get the Hadoop FileSystem
    fs = spark.sparkContext._jvm.org.apache.hadoop.fs.FileSystem.get(
        spark.sparkContext._jsc.hadoopConfiguration())
    
    # List files in the root directory
    root_status = fs.listStatus(spark.sparkContext._jvm.org.apache.hadoop.fs.Path("/"))
    
    print("Files in HDFS root directory:")
    for status in root_status:
        path = status.getPath().toString()
        print(f"  {path}")
        
    # Now try to list your data directory (where your CDR data should be)
    data_path = spark.sparkContext._jvm.org.apache.hadoop.fs.Path("/data")
    if fs.exists(data_path):
        data_status = fs.listStatus(data_path)
        print("\nFiles in /data directory:")
        for status in data_status:
            path = status.getPath().toString()
            print(f"  {path}")
    else:
        print("\nThe /data directory doesn't exist in HDFS.")
        
except Exception as e:
    print(f"Error accessing HDFS: {e}")

Files in HDFS root directory:
  hdfs://namenode:9000/tmp
  hdfs://namenode:9000/user

The /data directory doesn't exist in HDFS.


In [5]:
# Explore the /cdr directory
try:
    cdr_path = spark.sparkContext._jvm.org.apache.hadoop.fs.Path("/cdr")
    fs = spark.sparkContext._jvm.org.apache.hadoop.fs.FileSystem.get(
        spark.sparkContext._jsc.hadoopConfiguration())
    
    cdr_status = fs.listStatus(cdr_path)
    print("Files in /cdr directory:")
    for status in cdr_status:
        path = status.getPath().toString()
        print(f"  {path}")
        
    # Check if there's a subdirectory like /cdr/data
    cdr_data_path = spark.sparkContext._jvm.org.apache.hadoop.fs.Path("/cdr/data")
    if fs.exists(cdr_data_path):
        cdr_data_status = fs.listStatus(cdr_data_path)
        print("\nFiles in /cdr/data directory:")
        for status in cdr_data_status:
            path = status.getPath().toString()
            print(f"  {path}")
    
    # Explore the /data/converted directory
    converted_path = spark.sparkContext._jvm.org.apache.hadoop.fs.Path("/data/converted")
    converted_status = fs.listStatus(converted_path)
    print("\nFiles in /data/converted directory:")
    for status in converted_status:
        path = status.getPath().toString()
        print(f"  {path}")
        
except Exception as e:
    print(f"Error exploring HDFS directories: {e}")

Error exploring HDFS directories: An error occurred while calling o110.listStatus.
: java.io.FileNotFoundException: File /cdr does not exist.
	at org.apache.hadoop.hdfs.DistributedFileSystem.listStatusInternal(DistributedFileSystem.java:1104)
	at org.apache.hadoop.hdfs.DistributedFileSystem.access$600(DistributedFileSystem.java:147)
	at org.apache.hadoop.hdfs.DistributedFileSystem$24.doCall(DistributedFileSystem.java:1175)
	at org.apache.hadoop.hdfs.DistributedFileSystem$24.doCall(DistributedFileSystem.java:1172)
	at org.apache.hadoop.fs.FileSystemLinkResolver.resolve(FileSystemLinkResolver.java:81)
	at org.apache.hadoop.hdfs.DistributedFileSystem.listStatus(DistributedFileSystem.java:1182)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at jav

In [6]:
# Read all CDR data files at once
cdr_df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("hdfs://namenode:9000/cdr/data/*.csv")

# Check the schema
print("CDR data schema:")
cdr_df.printSchema()

# Check the count of records
print(f"\nTotal number of records: {cdr_df.count()}")

# Display a sample of the data
print("\nSample data:")
cdr_df.show(5, truncate=False)

AnalysisException: [PATH_NOT_FOUND] Path does not exist: hdfs://namenode:9000/cdr/data/*.csv.

25/05/07 19:09:38 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
25/05/07 19:09:38 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exiting due to error from cluster scheduler: Master removed our application: KILLED
	at org.apache.spark.errors.SparkCoreErrors$.clusterSchedulerError(SparkCoreErrors.scala:291)
	at org.apache.spark.scheduler.TaskSchedulerImpl.error(TaskSchedulerImpl.scala:981)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.dead(StandaloneSchedulerBackend.scala:165)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint.markDead(StandaloneAppClient.scala:263)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anonfun$receive$1.applyOrElse(StandaloneAppClient.scala:170)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:115)
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:213)
	at org.apache.spark.rpc.netty.Inbox.proce

In [23]:
# Read all CDR data files at once 
cdr_df = spark.read \
.option("header", "true") \
.option("inferSchema" , "true")\
.csv("hdfs://namenode:9000/cdr/data/*csv")

# Check the schema 
print("CDR data schema:")
cdr_df.printSchema()

# Check the count of records
print(f"\nTotal number of records: {cdr_df.count()}")
    

CDR data schema:
root
 |-- CDR_ID: long (nullable = true)
 |-- CDR_SUB_ID: integer (nullable = true)
 |-- CDR_TYPE: string (nullable = true)
 |-- SPLIT_CDR_REASON: string (nullable = true)
 |-- CDR_BATCH_ID: long (nullable = true)
 |-- SRC_REC_LINE_NO: integer (nullable = true)
 |-- SRC_CDR_ID: integer (nullable = true)
 |-- SRC_CDR_NO: long (nullable = true)
 |-- STATUS: string (nullable = true)
 |-- RE_RATING_TIMES: integer (nullable = true)
 |-- CREATE_DATE: long (nullable = true)
 |-- START_DATE: long (nullable = true)
 |-- END_DATE: long (nullable = true)
 |-- CUST_LOCAL_START_DATE: long (nullable = true)
 |-- CUST_LOCAL_END_DATE: long (nullable = true)
 |-- STD_EVT_TYPE_ID: integer (nullable = true)
 |-- EVT_SOURCE_CATEGORY: string (nullable = true)
 |-- OBJ_TYPE: string (nullable = true)
 |-- OBJ_ID: long (nullable = true)
 |-- OWNER_CUST_ID: long (nullable = true)
 |-- DEFAULT_ACCT_ID: long (nullable = true)
 |-- PRI_IDENTITY: integer (nullable = true)
 |-- BILL_CYCLE_ID: integ

### Cleaning CDR Data 

In [ ]:
# Check if we can access HDFS through Spark
try:
    # List files in HDFS
    files = spark.sparkContext._jvm.org.apache.hadoop.fs.FileSystem.get(
        spark.sparkContext._jsc.hadoopConfiguration()).listStatus(
        spark.sparkContext._jvm.org.apache.hadoop.fs.Path("/"))
    
    print("\nHDFS root directory contents:")
    for file in files:
        print(f"  {file.getPath().getName()}")
        
except Exception as e:
    print(f"\nError accessing HDFS through Spark: {e}")

# Add project root to Python path for imports
current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, "../.."))
sys.path.append(project_root)

# Import backup utilities
try:
    from src.utils.backup_utils import backup_hdfs_to_local, backup_hdfs_to_hdfs
except ImportError:
    print("Warning: Couldn't import backup utilities. Proceeding without backup.")
    
    def backup_hdfs_to_local(src, dst):
        print(f"Skipping local backup from {src} to {dst}")
        return None
        
    def backup_hdfs_to_hdfs(src, dst):
        print(f"Skipping HDFS backup from {src} to {dst}")
        return None

# Source and backup paths
hdfs_source_path = "/cdr/data"
local_backup_dir = "D:/network_trend_analysis/backups/cdr_data"
hdfs_backup_prefix = "/user/backup/cdr_data/backup"

# Perform backups before processing
print("Creating backups before processing data...")
hdfs_backup_path = backup_hdfs_to_hdfs(hdfs_source_path, hdfs_backup_prefix)
local_backup_path = backup_hdfs_to_local(hdfs_source_path, local_backup_dir)

if not (hdfs_backup_path or local_backup_path):
    print("WARNING: All backups failed. Do you want to continue anyway? (y/n)")
    response = input().strip().lower()
    if response != 'y':
        print("Aborting data processing.")
        sys.exit(1)


In [18]:
import os
import sys
import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Create Spark session with HDFS configuration
spark = SparkSession.builder \
    .appName("CDR-Analysis") \
    .config("spark.master", "spark://spark-master:7077") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://namenode:9000") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g") \
    .getOrCreate()

print("Spark session initialized.")

# Print Spark configuration for troubleshooting
print("Spark version:", spark.version)
print("\nSpark configuration:")
for item in spark.sparkContext.getConf().getAll():
    print(f"  {item[0]}: {item[1]}")


# Define constants for Algerian telecom data
# Wilaya prefix mapping with format validation
wilaya_prefix_map = {
    'Algiers': ['021', '023', '044'],  # First prefix is for ADSL, second is for FTTH
    'Oran': ['041', '042'],
    'Constantine': ['031', '043'],
    'Annaba': ['038', '045'],
    'Bejaia': ['034', '046'],
    'Tizi Ouzou': ['026', '047'],
    'Blida': ['025', '048'],
    'Setif': ['036', '049'],
    'Batna': ['033', '039'],
    'Tlemcen': ['043', '032'],
    'Mostaganem': ['045', '037'],
    'Boumerdas': ['024', '050'],
}

# Mobile operator prefixes - ensuring format 0yxx
mobile_operator_prefixes = {
    'Ooredoo': ['055', '056'],  # 0550, 0560 format
    'Mobilis': ['066', '067'],  # 0660, 0661 format
    'Djezzy': ['077', '078'],   # 0770, 0771, 0772 format
}

# ADSL and FTTH prefixes - ensuring landline format 0yy
adsl_prefixes = ["021", "023", "041", "031", "038", "034", "026", "025", "036", "033", "043", "045"]
ftth_prefixes = ["044", "042", "043", "045", "046", "047", "048", "049", "032", "037", "039", "050"]

# Call types
call_types = {
    1: "VOICE",
    2: "SMS",
    3: "DATA",
    4: "MMS",
    5: "VOICE_ROAMING",
    6: "SMS_ROAMING",
    7: "DATA_ROAMING",
    8: "MMS_ROAMING"
}
print("Loading CDR data...")

# Load the CDR data
cdr_df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv(f"hdfs://namenode:9000{hdfs_source_path}/*csv")

print(f"Loaded {cdr_df.count()} CDR records.")

# Function to convert epoch milliseconds to datetime with proper format
def convert_epoch_to_datetime(epoch_time):
    try:
        if epoch_time is not None and epoch_time > 0:
            dt = datetime.datetime.fromtimestamp(epoch_time/1000)
            # Format as DD/MM/YY HH:MM:SS
            return dt.strftime("%d/%m/%Y %H:%M:%S")
        return None
    except Exception as e:
        print(f"Error converting epoch time: {e}")
        return None

# Register UDF
convert_epoch_udf = udf(convert_epoch_to_datetime, StringType())

# Determine the line type based on Algerian phone number format
def get_line_type(phone_number):
    if phone_number is None:
        return "Unknown"
    
    # Convert to string and ensure it's properly formatted
    try:
        num_str = str(int(phone_number))
    except:
        return "Invalid"
    
    # Check for emergency numbers
    for service, number in emergency_numbers.items():
        if num_str.endswith(number):
            return f"Emergency-{service}"
    
    # Standardize format for analysis
    if num_str.startswith('00'):
        # International format with 00 prefix
        return "International"
    elif num_str.startswith('213'):
        # Algerian number with country code
        stripped_num = num_str[3:]
    elif num_str.startswith('0'):
        # Algerian number with leading 0
        stripped_num = num_str[1:]
    else:
        stripped_num = num_str
    
    # Now determine type based on number format
    
    # VoIP numbers (format 09x)
    if stripped_num.startswith('9') and len(stripped_num) == 8:
        return "VoIP"
    
    # Mobile numbers (format 0yxx - 9 digits including leading 0)
    if len(stripped_num) == 8:  # 9 digits with the leading 0
        prefix = stripped_num[0:2]
        if prefix in ['55', '56']:
            return "Mobile-Ooredoo"
        elif prefix in ['66', '67']:
            return "Mobile-Mobilis" 
        elif prefix in ['77', '78']:
            return "Mobile-Djezzy"
    
    # Fixed lines (format 0yy - 8 digits including leading 0)
    if len(stripped_num) == 7:  # 8 digits with the leading 0
        prefix = stripped_num[0:2]
        if prefix in [p[1:3] for p in adsl_prefixes]:
            return "ADSL"
        elif prefix in [p[1:3] for p in ftth_prefixes]:
            return "FTTH"
    
    return "Other"

# Determine wilaya based on phone number prefix
def get_wilaya_from_number(phone_number):
    if phone_number is None:
        return "Unknown"
    
    # Convert to string and ensure it's properly formatted
    try:
        num_str = str(int(phone_number))
    except:
        return "Unknown"
    
    # For fixed lines in Algeria (format 0yy)
    if len(num_str) >= 8:
        # Standardize format
        if num_str.startswith('0'):
            prefix = num_str[1:3]  # Get area code after leading 0
        elif num_str.startswith('213'):
            prefix = num_str[3:5]  # Get area code after 213 country code
        else:
            return "Unknown"
        
        # Look up wilaya based on prefix
        for wilaya, prefixes in wilaya_prefix_map.items():
            for p in prefixes:
                if p.endswith(prefix):
                    return wilaya
    
    return "Unknown"

# Get operator based on mobile number
def get_operator(phone_number):
    if phone_number is None:
        return "Unknown"
    
    # Convert to string and ensure it's properly formatted
    try:
        num_str = str(int(phone_number))
    except:
        return "Unknown"
    
    # For mobile numbers in Algeria (format 0yxx)
    if len(num_str) >= 9:  # Mobile numbers have 9 digits
        # Standardize format to get the operator prefix
        if num_str.startswith('0'):
            prefix = num_str[0:3]  # First 3 digits including leading 0
        elif num_str.startswith('213'):
            prefix = '0' + num_str[3:5]  # Convert to local format
        else:
            return "Unknown"
        
        # Check operator prefixes
        for operator, prefixes in mobile_operator_prefixes.items():
            for p in prefixes:
                if prefix.startswith('0' + p):
                    return operator
    
    return "Unknown"

# Map call type to descriptive name
def map_call_type(call_type_id):
    if call_type_id is None:
        return "Unknown"
    return call_types.get(int(call_type_id), "Other")

# Register UDFs
line_type_udf = udf(get_line_type, StringType())
wilaya_udf = udf(get_wilaya_from_number, StringType())
operator_udf = udf(get_operator, StringType())
call_type_udf = udf(map_call_type, StringType())

print("Transforming data...")

# Select and transform essential columns
clean_df = cdr_df.select(
    col("CDR_ID").alias("record_id"),
    convert_epoch_udf(col("START_DATE")).alias("call_start_time"),
    convert_epoch_udf(col("END_DATE")).alias("call_end_time"),
    col("CallingPartyNumber").alias("source_number"),
    col("CalledPartyNumber").alias("destination_number"),
    col("ACTUAL_USAGE").alias("duration_seconds"),
    round(col("ACTUAL_USAGE")/60, 2).alias("duration_minutes"),
    col("CallType").alias("call_type_id"),
    call_type_udf(col("CallType")).alias("call_type"),
    col("RoamState").alias("roaming_status"),
    col("DEBIT_AMOUNT").alias("charge_amount"),
    col("CallingCellID").alias("source_cell_id"),
    col("TerminationReason").alias("termination_reason"),
    col("IMEI").alias("device_id"),
    col("ServiceType").alias("service_type"),
    col("PayType").alias("payment_type"),
    col("CallingHomeCountryCode").alias("source_country_code"),
    col("CalledHomeCountryCode").alias("destination_country_code"),
    col("CallingHomeAreaNumber").alias("source_area_code"),
    col("CalledHomeAreaNumber").alias("destination_area_code"),
    col("MSCAddress").alias("switch_address"),
    col("BrandID").alias("brand_id"),
    col("MainOfferingID").alias("offering_id")
)

print("Enriching data with date components and call success flag...")

# Extract date components from the formatted timestamp
enriched_df = clean_df.withColumn(
    "call_date", to_date(to_timestamp(col("call_start_time"), "dd/MM/yyyy HH:mm:ss"))
).withColumn(
    "call_day", date_format(to_timestamp(col("call_start_time"), "dd/MM/yyyy HH:mm:ss"), "d")
).withColumn(
    "call_month", date_format(to_timestamp(col("call_start_time"), "dd/MM/yyyy HH:mm:ss"), "M")
).withColumn(
    "call_year", date_format(to_timestamp(col("call_start_time"), "dd/MM/yyyy HH:mm:ss"), "yyyy")
).withColumn(
    "call_hour", date_format(to_timestamp(col("call_start_time"), "dd/MM/yyyy HH:mm:ss"), "H")
).withColumn(
    "call_day_of_week", date_format(to_timestamp(col("call_start_time"), "dd/MM/yyyy HH:mm:ss"), "EEEE")
).withColumn(
    "call_success", when(col("termination_reason") == 0, 1).otherwise(0)
)

print("Adding Algerian telecom-specific enrichment...")

# Add enrichment data specific to Algerian telecom
final_df = enriched_df.withColumn(
    "source_line_type", line_type_udf(col("source_number"))
).withColumn(
    "destination_line_type", line_type_udf(col("destination_number"))
).withColumn(
    "source_wilaya", wilaya_udf(col("source_number"))
).withColumn(
    "destination_wilaya", wilaya_udf(col("destination_number"))
).withColumn(
    "source_operator", operator_udf(col("source_number"))
).withColumn(
    "destination_operator", operator_udf(col("destination_number"))
)

print("Removing duplicates and invalid records...")

# Remove duplicates and invalid records
valid_df = final_df.dropDuplicates(["record_id"]).filter(
    (col("call_start_time").isNotNull()) & 
    (col("source_number").isNotNull()) &
    (col("duration_seconds") > 0)
)

# Print analytics about the data quality
print(f"Initial record count: {cdr_df.count()}")
print(f"Final valid record count: {valid_df.count()}")
print(f"Removed {cdr_df.count() - valid_df.count()} invalid or duplicate records")

# Data quality report
print("\nData Quality Report:")
total_records = valid_df.count()

# Check phone number quality
source_line_types = valid_df.groupBy("source_line_type").count().withColumn(
    "percentage", round(col("count") * 100 / total_records, 2)
).orderBy(desc("count"))

print("\nSource Line Type Distribution:")
source_line_types.show(truncate=False)

# Check wilaya distribution
wilaya_dist = valid_df.groupBy("source_wilaya").count().withColumn(
    "percentage", round(col("count") * 100 / total_records, 2)
).orderBy(desc("count"))

print("\nSource Wilaya Distribution:")
wilaya_dist.show(truncate=False)

# Check operator distribution
operator_dist = valid_df.groupBy("source_operator").count().withColumn(
    "percentage", round(col("count") * 100 / total_records, 2)
).orderBy(desc("count"))

print("\nSource Operator Distribution:")
operator_dist.show(truncate=False)

print("\nSaving the cleaned and enriched data...")

# Save the cleaned and enriched data
valid_df.write.mode("overwrite").parquet("hdfs://namenode:9000/cdr/cleaned_data")

print("\nData cleaning and transformation complete!")
print(f"Cleaned data saved to: hdfs://namenode:9000/cdr/cleaned_data")

25/04/19 01:46:04 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
Spark session initialized.
Spark version: 3.3.4

Spark configuration:
  spark.executor.memory: 2g
  spark.app.submitTime: 1745023947046
  spark.driver.host: 9b90dc4a8722
  spark.hadoop.fs.defaultFS: hdfs://namenode:9000
  spark.executor.id: driver
  spark.app.name: CDR-Analysis
  spark.app.id: app-20250419005228-0000
  spark.app.startTime: 1745023947158
  spark.driver.extraJavaOptions: -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/

Loaded 89911 CDR records.


Py4JJavaError: An error occurred while calling o1330.get.
: java.util.NoSuchElementException: spark.sql.execution.pythonUDF.arrow.enabled
	at org.apache.spark.sql.errors.QueryExecutionErrors$.noSuchElementExceptionError(QueryExecutionErrors.scala:1678)
	at org.apache.spark.sql.internal.SQLConf.$anonfun$getConfString$3(SQLConf.scala:4587)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.internal.SQLConf.getConfString(SQLConf.scala:4587)
	at org.apache.spark.sql.RuntimeConfig.get(RuntimeConfig.scala:72)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:566)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(Reflect

In [18]:
import os
import sys
import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Create Spark session with HDFS configuration
spark = SparkSession.builder \
    .appName("CDR-Analysis") \
    .config("spark.master", "spark://spark-master:7077") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://namenode:9000") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g") \
    .getOrCreate()

print("Spark session initialized.")

# Print Spark configuration for troubleshooting
print("Spark version:", spark.version)
print("\nSpark configuration:")
for item in spark.sparkContext.getConf().getAll():
    print(f"  {item[0]}: {item[1]}")


# Define constants for Algerian telecom data
# Wilaya prefix mapping with format validation
wilaya_prefix_map = {
    'Algiers': ['021', '023', '044'],  # First prefix is for ADSL, second is for FTTH
    'Oran': ['041', '042'],
    'Constantine': ['031', '043'],
    'Annaba': ['038', '045'],
    'Bejaia': ['034', '046'],
    'Tizi Ouzou': ['026', '047'],
    'Blida': ['025', '048'],
    'Setif': ['036', '049'],
    'Batna': ['033', '039'],
    'Tlemcen': ['043', '032'],
    'Mostaganem': ['045', '037'],
    'Boumerdas': ['024', '050'],
}

# Mobile operator prefixes - ensuring format 0yxx
mobile_operator_prefixes = {
    'Ooredoo': ['055', '056'],  # 0550, 0560 format
    'Mobilis': ['066', '067'],  # 0660, 0661 format
    'Djezzy': ['077', '078'],   # 0770, 0771, 0772 format
}

# ADSL and FTTH prefixes - ensuring landline format 0yy
adsl_prefixes = ["021", "023", "041", "031", "038", "034", "026", "025", "036", "033", "043", "045"]
ftth_prefixes = ["044", "042", "043", "045", "046", "047", "048", "049", "032", "037", "039", "050"]

# VoIP prefixes - format 09x
voip_prefixes = ["09"]

# Emergency numbers
emergency_numbers = {
    "Police": "1548",
    "Fire": "14",
    "Gendarme": "1055",
    "Coast Guard": "1054",
}

# Call types
call_types = {
    1: "VOICE",
    2: "SMS",
    3: "DATA",
    4: "MMS",
    5: "VOICE_ROAMING",
    6: "SMS_ROAMING",
    7: "DATA_ROAMING",
    8: "MMS_ROAMING"
}
print("Loading CDR data...")

# Load the CDR data
cdr_df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv(f"hdfs://namenode:9000{hdfs_source_path}/*csv")

print(f"Loaded {cdr_df.count()} CDR records.")

# Function to convert epoch milliseconds to datetime with proper format
def convert_epoch_to_datetime(epoch_time):
    try:
        if epoch_time is not None and epoch_time > 0:
            dt = datetime.datetime.fromtimestamp(epoch_time/1000)
            # Format as DD/MM/YY HH:MM:SS
            return dt.strftime("%d/%m/%Y %H:%M:%S")
        return None
    except Exception as e:
        print(f"Error converting epoch time: {e}")
        return None

# Register UDF
convert_epoch_udf = udf(convert_epoch_to_datetime, StringType())

# Determine the line type based on Algerian phone number format
def get_line_type(phone_number):
    if phone_number is None:
        return "Unknown"
    
    # Convert to string and ensure it's properly formatted
    try:
        num_str = str(int(phone_number))
    except:
        return "Invalid"
    
    # Check for emergency numbers
    for service, number in emergency_numbers.items():
        if num_str.endswith(number):
            return f"Emergency-{service}"
    
    # Standardize format for analysis
    if num_str.startswith('00'):
        # International format with 00 prefix
        return "International"
    elif num_str.startswith('213'):
        # Algerian number with country code
        stripped_num = num_str[3:]
    elif num_str.startswith('0'):
        # Algerian number with leading 0
        stripped_num = num_str[1:]
    else:
        stripped_num = num_str
    
    # Now determine type based on number format
    
    # VoIP numbers (format 09x)
    if stripped_num.startswith('9') and len(stripped_num) == 8:
        return "VoIP"
    
    # Mobile numbers (format 0yxx - 9 digits including leading 0)
    if len(stripped_num) == 8:  # 9 digits with the leading 0
        prefix = stripped_num[0:2]
        if prefix in ['55', '56']:
            return "Mobile-Ooredoo"
        elif prefix in ['66', '67']:
            return "Mobile-Mobilis" 
        elif prefix in ['77', '78']:
            return "Mobile-Djezzy"
    
    # Fixed lines (format 0yy - 8 digits including leading 0)
    if len(stripped_num) == 7:  # 8 digits with the leading 0
        prefix = stripped_num[0:2]
        if prefix in [p[1:3] for p in adsl_prefixes]:
            return "ADSL"
        elif prefix in [p[1:3] for p in ftth_prefixes]:
            return "FTTH"
    
    return "Other"

# Determine wilaya based on phone number prefix
def get_wilaya_from_number(phone_number):
    if phone_number is None:
        return "Unknown"
    
    # Convert to string and ensure it's properly formatted
    try:
        num_str = str(int(phone_number))
    except:
        return "Unknown"
    
    # For fixed lines in Algeria (format 0yy)
    if len(num_str) >= 8:
        # Standardize format
        if num_str.startswith('0'):
            prefix = num_str[1:3]  # Get area code after leading 0
        elif num_str.startswith('213'):
            prefix = num_str[3:5]  # Get area code after 213 country code
        else:
            return "Unknown"
        
        # Look up wilaya based on prefix
        for wilaya, prefixes in wilaya_prefix_map.items():
            for p in prefixes:
                if p.endswith(prefix):
                    return wilaya
    
    return "Unknown"

# Get operator based on mobile number
def get_operator(phone_number):
    if phone_number is None:
        return "Unknown"
    
    # Convert to string and ensure it's properly formatted
    try:
        num_str = str(int(phone_number))
    except:
        return "Unknown"
    
    # For mobile numbers in Algeria (format 0yxx)
    if len(num_str) >= 9:  # Mobile numbers have 9 digits
        # Standardize format to get the operator prefix
        if num_str.startswith('0'):
            prefix = num_str[0:3]  # First 3 digits including leading 0
        elif num_str.startswith('213'):
            prefix = '0' + num_str[3:5]  # Convert to local format
        else:
            return "Unknown"
        
        # Check operator prefixes
        for operator, prefixes in mobile_operator_prefixes.items():
            for p in prefixes:
                if prefix.startswith('0' + p):
                    return operator
    
    return "Unknown"

# Map call type to descriptive name
def map_call_type(call_type_id):
    if call_type_id is None:
        return "Unknown"
    return call_types.get(int(call_type_id), "Other")

# Register UDFs
line_type_udf = udf(get_line_type, StringType())
wilaya_udf = udf(get_wilaya_from_number, StringType())
operator_udf = udf(get_operator, StringType())
call_type_udf = udf(map_call_type, StringType())

print("Transforming data...")

# Select and transform essential columns
clean_df = cdr_df.select(
    col("CDR_ID").alias("record_id"),
    convert_epoch_udf(col("START_DATE")).alias("call_start_time"),
    convert_epoch_udf(col("END_DATE")).alias("call_end_time"),
    col("CallingPartyNumber").alias("source_number"),
    col("CalledPartyNumber").alias("destination_number"),
    col("ACTUAL_USAGE").alias("duration_seconds"),
    round(col("ACTUAL_USAGE")/60, 2).alias("duration_minutes"),
    col("CallType").alias("call_type_id"),
    call_type_udf(col("CallType")).alias("call_type"),
    col("RoamState").alias("roaming_status"),
    col("DEBIT_AMOUNT").alias("charge_amount"),
    col("CallingCellID").alias("source_cell_id"),
    col("TerminationReason").alias("termination_reason"),
    col("IMEI").alias("device_id"),
    col("ServiceType").alias("service_type"),
    col("PayType").alias("payment_type"),
    col("CallingHomeCountryCode").alias("source_country_code"),
    col("CalledHomeCountryCode").alias("destination_country_code"),
    col("CallingHomeAreaNumber").alias("source_area_code"),
    col("CalledHomeAreaNumber").alias("destination_area_code"),
    col("MSCAddress").alias("switch_address"),
    col("BrandID").alias("brand_id"),
    col("MainOfferingID").alias("offering_id")
)

print("Enriching data with date components and call success flag...")

# Extract date components from the formatted timestamp
enriched_df = clean_df.withColumn(
    "call_date", to_date(to_timestamp(col("call_start_time"), "dd/MM/yyyy HH:mm:ss"))
).withColumn(
    "call_day", date_format(to_timestamp(col("call_start_time"), "dd/MM/yyyy HH:mm:ss"), "d")
).withColumn(
    "call_month", date_format(to_timestamp(col("call_start_time"), "dd/MM/yyyy HH:mm:ss"), "M")
).withColumn(
    "call_year", date_format(to_timestamp(col("call_start_time"), "dd/MM/yyyy HH:mm:ss"), "yyyy")
).withColumn(
    "call_hour", date_format(to_timestamp(col("call_start_time"), "dd/MM/yyyy HH:mm:ss"), "H")
).withColumn(
    "call_day_of_week", date_format(to_timestamp(col("call_start_time"), "dd/MM/yyyy HH:mm:ss"), "EEEE")
).withColumn(
    "call_success", when(col("termination_reason") == 0, 1).otherwise(0)
)

print("Adding Algerian telecom-specific enrichment...")

# Add enrichment data specific to Algerian telecom
final_df = enriched_df.withColumn(
    "source_line_type", line_type_udf(col("source_number"))
).withColumn(
    "destination_line_type", line_type_udf(col("destination_number"))
).withColumn(
    "source_wilaya", wilaya_udf(col("source_number"))
).withColumn(
    "destination_wilaya", wilaya_udf(col("destination_number"))
).withColumn(
    "source_operator", operator_udf(col("source_number"))
).withColumn(
    "destination_operator", operator_udf(col("destination_number"))
)

print("Removing duplicates and invalid records...")

# Remove duplicates and invalid records
valid_df = final_df.dropDuplicates(["record_id"]).filter(
    (col("call_start_time").isNotNull()) & 
    (col("source_number").isNotNull()) &
    (col("duration_seconds") > 0)
)

# Print analytics about the data quality
print(f"Initial record count: {cdr_df.count()}")
print(f"Final valid record count: {valid_df.count()}")
print(f"Removed {cdr_df.count() - valid_df.count()} invalid or duplicate records")

# Data quality report
print("\nData Quality Report:")
total_records = valid_df.count()

# Check phone number quality
source_line_types = valid_df.groupBy("source_line_type").count().withColumn(
    "percentage", round(col("count") * 100 / total_records, 2)
).orderBy(desc("count"))

print("\nSource Line Type Distribution:")
source_line_types.show(truncate=False)

# Check wilaya distribution
wilaya_dist = valid_df.groupBy("source_wilaya").count().withColumn(
    "percentage", round(col("count") * 100 / total_records, 2)
).orderBy(desc("count"))

print("\nSource Wilaya Distribution:")
wilaya_dist.show(truncate=False)

# Check operator distribution
operator_dist = valid_df.groupBy("source_operator").count().withColumn(
    "percentage", round(col("count") * 100 / total_records, 2)
).orderBy(desc("count"))

print("\nSource Operator Distribution:")
operator_dist.show(truncate=False)

print("\nSaving the cleaned and enriched data...")

# Save the cleaned and enriched data
valid_df.write.mode("overwrite").parquet("hdfs://namenode:9000/cdr/cleaned_data")

print("\nData cleaning and transformation complete!")
print(f"Cleaned data saved to: hdfs://namenode:9000/cdr/cleaned_data")

25/04/19 01:46:04 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
Spark session initialized.
Spark version: 3.3.4

Spark configuration:
  spark.executor.memory: 2g
  spark.app.submitTime: 1745023947046
  spark.driver.host: 9b90dc4a8722
  spark.hadoop.fs.defaultFS: hdfs://namenode:9000
  spark.executor.id: driver
  spark.app.name: CDR-Analysis
  spark.app.id: app-20250419005228-0000
  spark.app.startTime: 1745023947158
  spark.driver.extraJavaOptions: -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/

Loaded 89911 CDR records.


Py4JJavaError: An error occurred while calling o1330.get.
: java.util.NoSuchElementException: spark.sql.execution.pythonUDF.arrow.enabled
	at org.apache.spark.sql.errors.QueryExecutionErrors$.noSuchElementExceptionError(QueryExecutionErrors.scala:1678)
	at org.apache.spark.sql.internal.SQLConf.$anonfun$getConfString$3(SQLConf.scala:4587)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.internal.SQLConf.getConfString(SQLConf.scala:4587)
	at org.apache.spark.sql.RuntimeConfig.get(RuntimeConfig.scala:72)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:566)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(Reflect

In [3]:
import sys
sys.path.append('/home/jovyan/work/spark-apps')

from spark_init import init_spark

spark = init_spark("CDR Anonymization in Notebook")


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/16 21:39:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


✅ SparkSession initialized (App: CDR Anonymization in Notebook, Spark: 3.5.1)


IllegalArgumentException: Error while instantiating 'org.apache.spark.sql.hive.HiveSessionStateBuilder':

JVM stacktrace:
java.lang.IllegalArgumentException: Error while instantiating 'org.apache.spark.sql.hive.HiveSessionStateBuilder':
	at org.apache.spark.sql.SparkSession$.org$apache$spark$sql$SparkSession$$instantiateSessionState(SparkSession.scala:1322)
	at org.apache.spark.sql.SparkSession.$anonfun$sessionState$2(SparkSession.scala:162)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.SparkSession.sessionState$lzycompute(SparkSession.scala:160)
	at org.apache.spark.sql.SparkSession.sessionState(SparkSession.scala:157)
	at org.apache.spark.sql.SparkSession.conf$lzycompute(SparkSession.scala:185)
	at org.apache.spark.sql.SparkSession.conf(SparkSession.scala:185)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.i

In [2]:
from anonymize_cdr import anonymize_cdr

input_path = "hdfs://namenode:9000/user/hive/warehouse/raw/*.csv"
output_path = "hdfs://namenode:9000/user/hive/warehouse/anonymized/"
salt = "random_salt_2025"
pii_columns = [
    "MSISDN", "IMSI", "IMEI", "CALLING_NBR", "CALLED_NBR",
    "FIXED_NUMBER", "MOBILE_NUMBER", "SUBSCRIBER_ID"
]

anonymize_cdr(input_path, output_path, salt, pii_columns)


ModuleNotFoundError: No module named 'spark_init'